
## Задача № 2

Шаг 1
Получить доступ к public data внутри BigQuery. 
Инструкция по ссылке: https://support.google.com/analytics/answer/7586738?hl=en#zippy=%2Cin-this-article 
Нас интересует набор данных google_analytics_sample и датасет ga_sessions. 

Шаг 2
Выгрузить таблицу из BQ (можно через сохранение на Google Drive или BQ API)
Набор полей в Select определить самостоятельно. Период запроса - 2 недели

Шаг 3
с помощью Python сформировать последовательности источников визитов пользователя, которые привели к транзакции.

- Источник определяем как связку utm_source + utm_medium. 
- Транзакция определяется, как hits.transaction.transactionId соответствует регулярному выражению .*\d+.*

Пример:
- Пользователь, который совершил визит с рекламы Яндекса, а затем с рекламы Google и совершил транзакцию будет иметь цепочку
yandex_cpc => google_cpc
- Пользователь совершил переход из поиска Google, а затем из поиска Яндекс. Транзакций не совершал. Цепочка для него отсутствует.

Если не совсем понятно, что такое последовательность касаний для каждого пользователя, то вот обучающее видео (https://www.youtube.com/watch?v=PW_9TMZKJp0). 

В результате ожидаем jupiter ноутбук с вашими комментариями и 2-мя результирующими таблицами:
1. Для каждой транзакции сформирована цепочка касаний
2. Топ-10 самых популярных цепочек касаний. Популярность определяется на основе количества таких цепочек в датасете.

Цепочка определяется так: берем юзера у которого был визит с транзакцией, его визиты и время визитов. Отсортироваваем таблицу сначала по юзерам, потом по времени начала визита. Все источники предшествующих до транзакции визитов (включая визит с транзакцией) определяем как цепочку, которая привела к транзакции. Если у пользователя было несколько транзакций, то вторая и все последующие цепочки будут начинаться со следующего визита после предыдущей транзакции. В реальных кейсах этот момент лучше обсуждать с заказчиком задачи, так как в таком случае метод определения цепочки для последующих транзакций может отличаться от случаю к случаю, все зависит от целей.


Задачу я решила двумя способами. 
1. Все основные расчеты проводятся в питоне. Минус этого метода в том, что для определения транзакции для каждого визита я использую циклы. Если данных очень много, то циклы будут работать медленно. Плюс в том, что сам алгоритм довольно простой и у меня ушло гораздо меньше времени на его реализацию, чем во втором методе. 
2. Определение номера транзакции для каждого визита определяется в SQL запросе. Плюс в том, что на SQL это будет работать быстрее. Минус в том, что запрос сложный и чтобы его написать мне понадобилось больше времени. 


### Сначала опишу первый способ. 


In [1]:
# импортируем необходимые библиотеки

import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account


In [2]:
# Данные для доступа в BQ 

credentials = service_account.Credentials.from_service_account_file(
    'abiding-casing-369322-32d08fafdf4a.json')
project_id = 'abiding-casing-369322'


Пишем простой запрос в SQL на диалекте BigQuery

Выгружаем: 
- userId - (fullvisitorid) - id юзера
- unixTime  -(visitStartTime) - время визита в формате unixtime
- visitId - id визита. Оказалось, что это то же самое что и время визита, по идее можно было оставить только одно из этих полей
- sourceType - связка source_medium
- trId - (hits.transaction.transactionId) - номер транзакции

Подключаемся к BQ и создаем датафрейм.
Для подключения нужен id проекта в BQ и ключ в виде JSON файла, генерируемый из BQ. 

Данные сразу отсортированы по времени визитов в обратном порядке, то есть от последнего к первому. Так удобнее было писать алгоритм для определения номера транзакции для визитов.

In [3]:
# Запрос с небольшими комментариями https://pastebin.com/w36sWYEM

query = """
with   

users_with_transaction as (
select 
  d.fullvisitorid,
  d.visitId,
  d.visitStartTime,
  hits.transaction.transactionId as trId
from 
  `bigquery-public-data.google_analytics_sample.ga_sessions_*` as d 
LEFT JOIN UNNEST(hits) as hits
where 
  hits.transaction.transactionId is not null
  and _TABLE_SUFFIX BETWEEN "20170801" and "20170814"
group by 
  1,2,3,4
),

visits as (
select 
  d.fullvisitorid as userId,
  d.visitStartTime as unixTime,
  d.visitId,
  concat(d.trafficSource.source, '_', d.trafficSource.medium) as sourceType, 
  u.trId 
from 
  `bigquery-public-data.google_analytics_sample.ga_sessions_*` as d 
LEFT JOIN UNNEST(hits) as hits
left join 
  users_with_transaction as u 
  on u.visitId = d.visitId
where 
  d.fullvisitorid in (select fullvisitorid from users_with_transaction) 
  and _TABLE_SUFFIX BETWEEN "20170801" and "20170814"
group by 
  1,2,3,4,5
)

select * from visits 
order by userId, unixTime desc
 """


df= pd.read_gbq(query, project_id=project_id, credentials=credentials)
df.head(20)




,userId,unixTime,visitId,sourceType,trId
0,0126612099954375841,1501606687,1501606687,(direct)_(none),ORD201708012550
1,0188308435688462855,1501651868,1501651868,(direct)_(none),ORD201708012558
2,024507252193437459,1501595478,1501595478,mail.google.com_referral,ORD201708012322
3,024507252193437459,1501594747,1501594747,(direct)_(none),None
4,03717831825503026,1501649573,1501649573,(direct)_(none),None
5,03717831825503026,1501641303,1501641303,(direct)_(none),ORD201708012493
6,0377506370870345639,1501637431,1501637431,(direct)_(none),None
7,0377506370870345639,1501608387,1501608387,(direct)_(none),ORD201708012447
8,0509972280802528263,1501621191,1501621191,(direct)_(none),ORD201708011814
9,0509972280802528263,1501620463,1501620463,(direct)_(none),None


### 1. Для каждой транзакции сформирована цепочка касаний

Порядок действий: 
1. Сначала создадим таблицу где в отдельном столбце пропишем номер транзакции для каждого предшествующего визита или визита в котором эта транзакция была совершена. 
2. Уберем визиты, которые не предшествовали транзакции. 
3. Сгруппируем номера транзакций и для каждой пропишем список источников всех предшествующих визитов 

Описание алгоритма для получения таблицы где для каждого визита прописана соответствующий номер транзакции: 

0. Имеем таблицу с визитами по юзерам, которые совершали хотя бы одну транзакцию. Визиты отсортированы в обратном порядке по времени (от последнего к первому).
1. Проходимся по каждому уникальному юзеру 
2. Запоминаем датафрейм с текущим юзером 
3. Проходимся по визитам в таблице с данными по текущему юзеру
4. Если находим визит с наличием транзакции,
5. то сохраняем номер этой транзакции в отдельную переменную
6. и в отдельном столбце прописываем номер этой транзакции для текущего визита
6. далее фиксируем новый датафрейм - это остаток таблицы начиная от визита в котором мы нашли эту транзакцию 
7. Проходимся далее по визитам "обрезанного" датафрейма 
8. И ищем в нем визиты в которых не было транзакции с помощью if - это и есть те визиты, которые происходили до совершения сохраненной транзакции конкретным пользователем
9. Делаем проверку на то, что текущий визит не является визитом с наличием транзакции 
10. Если транзакции нет, то считаем что этот визит предшетствовал сохраненной транзакции и записываем номер этой транзакции в отдельный столбец 
11. Если находим визит с новой транзакцией, то пропускаем этот визит и идем дальше, относя все остальные визиты к последней сохраненной в предыдущем цикле транзакции 
12. Таким образом, может получится так, что в случае, если у юзера будет найдена новая транзакция, то алгоритм запишет неверный номер последней сохраненной транзакции, ведь другие визиты относятся уже к другой транзакции, которую мы пропустили
13. Но по мере прохождения цикла и сохранению новых транзакци у юзера, данные в таблице перезапишутся и все номера транзакций будут распределены по нужным визитам.Таким образом, имеем таблицу, в которой для каждого визита прописана транзакция которая следует за этим визитом или была произведена в этом же визите, столбец trId_gr 


In [4]:
for user in df['userId'].unique():
    df1 = df[df['userId']==user]
    for i in df1.index: 
        if df1['trId'].isna()[i] == False: 
            current_transaction = df.loc[i,'trId']
            df.loc[i,'trId_gr'] = current_transaction 
            df2 = df1.loc[i:] 
            for j in df2.index: 
                if df2['trId'].isna()[j]== True: 
                    df.loc[j,'trId_gr'] = current_transaction
                else:
                    continue

display(df.head(20))          


,userId,unixTime,visitId,sourceType,trId,trId_gr
0,0126612099954375841,1501606687,1501606687,(direct)_(none),ORD201708012550,ORD201708012550
1,0188308435688462855,1501651868,1501651868,(direct)_(none),ORD201708012558,ORD201708012558
2,024507252193437459,1501595478,1501595478,mail.google.com_referral,ORD201708012322,ORD201708012322
3,024507252193437459,1501594747,1501594747,(direct)_(none),None,ORD201708012322
4,03717831825503026,1501649573,1501649573,(direct)_(none),None,NaN
5,03717831825503026,1501641303,1501641303,(direct)_(none),ORD201708012493,ORD201708012493
6,0377506370870345639,1501637431,1501637431,(direct)_(none),None,NaN
7,0377506370870345639,1501608387,1501608387,(direct)_(none),ORD201708012447,ORD201708012447
8,0509972280802528263,1501621191,1501621191,(direct)_(none),ORD201708011814,ORD201708011814
9,0509972280802528263,1501620463,1501620463,(direct)_(none),None,ORD201708011814


Готово. 

Теперь избавимся от ненужных визитов и отсортируем датафрейм в последовательном порядке 

In [5]:
# Избавляемся от ненужных визитов, которые не предшетствовали транзакциям 
df = df[~df['trId_gr'].isna()]

# сортируем датафрейм в правильном порядке (от первого визита к последнему)
df = df.sort_values(['userId','unixTime'], ascending=True)
display(df.head(20))


,userId,unixTime,visitId,sourceType,trId,trId_gr
0,0126612099954375841,1501606687,1501606687,(direct)_(none),ORD201708012550,ORD201708012550
1,0188308435688462855,1501651868,1501651868,(direct)_(none),ORD201708012558,ORD201708012558
3,024507252193437459,1501594747,1501594747,(direct)_(none),None,ORD201708012322
2,024507252193437459,1501595478,1501595478,mail.google.com_referral,ORD201708012322,ORD201708012322
5,03717831825503026,1501641303,1501641303,(direct)_(none),ORD201708012493,ORD201708012493
7,0377506370870345639,1501608387,1501608387,(direct)_(none),ORD201708012447,ORD201708012447
9,0509972280802528263,1501620463,1501620463,(direct)_(none),None,ORD201708011814
8,0509972280802528263,1501621191,1501621191,(direct)_(none),ORD201708011814,ORD201708011814
10,0815047945706399620,1501612980,1501612980,(direct)_(none),ORD201708012428,ORD201708012428
11,0843143557737586340,1501614315,1501614315,(direct)_(none),ORD201708012538,ORD201708012538


Теперь прогруппируем получившуюся таблицу по номерам транзакций и запишем для каждой транзакции последовательность источников, которые привели к транзакции

In [6]:
sources_list_df = df.groupby('trId_gr')['sourceType'].apply(list).reset_index().rename(columns = {'trId_gr':'transaction_id', 'sourceType': 'sources_list'})
sources_list_df['sources_list'] = sources_list_df['sources_list'].apply(lambda x: " => ".join(x))

display(sources_list_df.head(20))




,transaction_id,sources_list
0,ORD201708011814,(direct)_(none) => (direct)_(none)
1,ORD201708012186,(direct)_(none) => (direct)_(none) => (direct)...
2,ORD201708012305,(direct)_(none)
3,ORD201708012322,(direct)_(none) => mail.google.com_referral
4,ORD201708012323,(direct)_(none) => (direct)_(none) => (direct)...
5,ORD201708012348,(direct)_(none) => (direct)_(none)
6,ORD201708012355,(direct)_(none) => (direct)_(none)
7,ORD201708012360,(direct)_(none)
8,ORD201708012371,(direct)_(none)
9,ORD201708012401,(direct)_(none) => (direct)_(none)


Первая результирующая таблица готова. 

transaction_id - номер транзакции

sources_list - список источников, которые привели к транзакции. Расположенные в последовательном порядке от первого визита до визита, в котором была транзакция.

### 2. Топ-10 самых популярных цепочек касаний. 
Популярность определяется на основе количества таких цепочек в датасете.

In [7]:
# группируем по количеству нахождений последовательностей источников по каждой последовательности
sources_list_series = sources_list_df.groupby('sources_list')['sources_list'].count()

# преобразуем результат в датафрейм для удобства 
sources_list_count_df = pd.DataFrame({'sources_list': sources_list_series.index, 'count': sources_list_series.values})

# отображаем отсортированный список 
display(sources_list_count_df.sort_values(['count'], ascending=False).head(10))




,sources_list,count
0,(direct)_(none),32
1,(direct)_(none) => (direct)_(none),8
2,(direct)_(none) => (direct)_(none) => (direct)...,2
3,(direct)_(none) => mail.google.com_referral,1


Вторая результирующая таблица готова. 

sources_row - список источников, которые привели к транзакции

count - частота встречаемости каждой цепочки в абсолютных значениях

### Итого: 

In [8]:

print('Первая таблица')
display(sources_list_df[['transaction_id', 'sources_list']])
print(' ')
print('Вторая таблица')
display(sources_list_count_df.sort_values(['count'], ascending=False).head(10))



Первая таблица


,transaction_id,sources_list
0,ORD201708011814,(direct)_(none) => (direct)_(none)
1,ORD201708012186,(direct)_(none) => (direct)_(none) => (direct)...
2,ORD201708012305,(direct)_(none)
3,ORD201708012322,(direct)_(none) => mail.google.com_referral
4,ORD201708012323,(direct)_(none) => (direct)_(none) => (direct)...
5,ORD201708012348,(direct)_(none) => (direct)_(none)
6,ORD201708012355,(direct)_(none) => (direct)_(none)
7,ORD201708012360,(direct)_(none)
8,ORD201708012371,(direct)_(none)
9,ORD201708012401,(direct)_(none) => (direct)_(none)


 
Вторая таблица


,sources_list,count
0,(direct)_(none),32
1,(direct)_(none) => (direct)_(none),8
2,(direct)_(none) => (direct)_(none) => (direct)...,2
3,(direct)_(none) => mail.google.com_referral,1


### Второй способ

С помощью SQL запроса сразу достаем таблицу в которой каждому предшетствующему транзакции визиту соответствует номер транзакции к которой эти визиты привели. 
Сразу очистим от визитов, которые не привели к транзакции, то есть не предшетствовали ей. 

In [9]:
# ссылка на запрос с комментариями https://pastebin.com/VXaEQCUn 

query1 = """
with   

users_with_transaction1 as (
select 
  d.fullvisitorid,
  d.visitId,
  d.visitStartTime,
  hits.transaction.transactionId as trId
from 
  `bigquery-public-data.google_analytics_sample.ga_sessions_*` as d 
LEFT JOIN UNNEST(hits) as hits
where 
  hits.transaction.transactionId is not null
  and _TABLE_SUFFIX BETWEEN "20170801" and "20170814"
group by 
  1,2,3,4
),


users_with_transaction as (
select 
  fullvisitorid,
  visitId,
  trId,
  row_number() over(partition by fullvisitorid order by visitStartTime) as tr_num
from 
  users_with_transaction1
),

visits as (
select 
  d.fullvisitorid as userId,
  d.visitStartTime as unixTime,
  d.visitId,
  concat(d.trafficSource.source, '_', d.trafficSource.medium) as sourceType, 
  u.trId, 
  u.tr_num
from 
  `bigquery-public-data.google_analytics_sample.ga_sessions_*` as d 
LEFT JOIN UNNEST(hits) as hits
left join 
  users_with_transaction as u 
  on u.visitId = d.visitId
where 
  d.fullvisitorid in (select fullvisitorid from users_with_transaction) 
  and _TABLE_SUFFIX BETWEEN "20170801" and "20170814"
group by 
  1,2,3,4,5,6
),

visits_1 as (
select 
  userId,
  visitId,
  unixTime,
  sourceType,
  trId,
  tr_num,
  min(tr_num) over(partition by userId order by unixTime asc ROWS BETWEEN CURRENT ROW and UNBOUNDED FOLLOWING) as tr_num_group
from 
  visits as v 
),

transaction_groups as (
select 
  userId,
  trId,
  tr_num
from 
  visits_1 
where 
  trId is not null
group by 
  1,2,3
)

select 
  v.userId,
  v.visitId,
  v.unixTime,
  v.sourceType,
  v.trId,
  v.tr_num,
  v.tr_num_group,
  t.trId as trId_group
from 
  visits_1 as v 
left join 
  transaction_groups as t    
  on t.userId = v.userId
  and t.tr_num = v.tr_num_group
where 
  v.tr_num_group is not null 
order by 
  1,3 """


df1= pd.read_gbq(query1, project_id=project_id, credentials=credentials)
display(df1.head(20))




,userId,visitId,unixTime,sourceType,trId,tr_num,tr_num_group,trId_group
0,0126612099954375841,1501606687,1501606687,(direct)_(none),ORD201708012550,1.0,1,ORD201708012550
1,0188308435688462855,1501651868,1501651868,(direct)_(none),ORD201708012558,1.0,1,ORD201708012558
2,024507252193437459,1501594747,1501594747,(direct)_(none),None,NaN,1,ORD201708012322
3,024507252193437459,1501595478,1501595478,mail.google.com_referral,ORD201708012322,1.0,1,ORD201708012322
4,03717831825503026,1501641303,1501641303,(direct)_(none),ORD201708012493,1.0,1,ORD201708012493
5,0377506370870345639,1501608387,1501608387,(direct)_(none),ORD201708012447,1.0,1,ORD201708012447
6,0509972280802528263,1501620463,1501620463,(direct)_(none),None,NaN,1,ORD201708011814
7,0509972280802528263,1501621191,1501621191,(direct)_(none),ORD201708011814,1.0,1,ORD201708011814
8,0815047945706399620,1501612980,1501612980,(direct)_(none),ORD201708012428,1.0,1,ORD201708012428
9,0843143557737586340,1501614315,1501614315,(direct)_(none),ORD201708012538,1.0,1,ORD201708012538


- userId - id юзера
- visitId - id визита
- unixTime - время визита в формате unixtime 
- sourceType - источник (source_medium)
- trId - id транзакции, присутствует только у тех визитов, где была транзакция
- tr_num - последовательный номер транзакции в разрезе юзера, присутствует только у тех визитов, где была транзакция
- tr_num_group - техническое поле 
- trId_group - номер транзакции, проставлен к каждому соответствующему визиту 
  
  
  
Далее алгоритм такой же как и в первом варианте решения. 

In [10]:
sources_list_df1 = df1.groupby('trId_group')['sourceType'].apply(list).reset_index().rename(columns = {'trId_group':'transaction_id', 'sourceType': 'sources_list'})
sources_list_df1['sources_list'] = sources_list_df1['sources_list'].apply(lambda x: " => ".join(x))
display(sources_list_df1.head(20))


,transaction_id,sources_list
0,ORD201708011814,(direct)_(none) => (direct)_(none)
1,ORD201708012186,(direct)_(none) => (direct)_(none) => (direct)...
2,ORD201708012305,(direct)_(none)
3,ORD201708012322,(direct)_(none) => mail.google.com_referral
4,ORD201708012323,(direct)_(none) => (direct)_(none) => (direct)...
5,ORD201708012348,(direct)_(none) => (direct)_(none)
6,ORD201708012355,(direct)_(none) => (direct)_(none)
7,ORD201708012360,(direct)_(none)
8,ORD201708012371,(direct)_(none)
9,ORD201708012401,(direct)_(none) => (direct)_(none)


In [11]:

sources_list_series1 = sources_list_df1.groupby('sources_list')['sources_list'].count()
sources_list_count_df1 = pd.DataFrame({'sources_list': sources_list_series1.index, 'count': sources_list_series1.values})

display(sources_list_count_df1.sort_values(['count'], ascending=False).head(10))




,sources_list,count
0,(direct)_(none),32
1,(direct)_(none) => (direct)_(none),8
2,(direct)_(none) => (direct)_(none) => (direct)...,2
3,(direct)_(none) => mail.google.com_referral,1


### Итого

In [12]:
print('Первая таблица')
display(sources_list_df1[['transaction_id', 'sources_list']])
print(' ')
print('Вторая таблица')
display(sources_list_count_df1.sort_values(['count'], ascending=False).head(10))




Первая таблица


,transaction_id,sources_list
0,ORD201708011814,(direct)_(none) => (direct)_(none)
1,ORD201708012186,(direct)_(none) => (direct)_(none) => (direct)...
2,ORD201708012305,(direct)_(none)
3,ORD201708012322,(direct)_(none) => mail.google.com_referral
4,ORD201708012323,(direct)_(none) => (direct)_(none) => (direct)...
5,ORD201708012348,(direct)_(none) => (direct)_(none)
6,ORD201708012355,(direct)_(none) => (direct)_(none)
7,ORD201708012360,(direct)_(none)
8,ORD201708012371,(direct)_(none)
9,ORD201708012401,(direct)_(none) => (direct)_(none)


 
Вторая таблица


,sources_list,count
0,(direct)_(none),32
1,(direct)_(none) => (direct)_(none),8
2,(direct)_(none) => (direct)_(none) => (direct)...,2
3,(direct)_(none) => mail.google.com_referral,1


Результаты получились одинаковые в обоих вариантах решения. 
